In [320]:
import numpy as np
import gurobipy as grb
import pandas as pd
from gurobipy import GRB
from collections import defaultdict
import gurobipy_pandas as gppd

In [321]:
df = pd.read_csv("data.csv")
df.Maximum_amount = df.Maximum_amount.fillna(GRB.INFINITY)
df

,Main food description,Category,Price,Maximum_amount,Calories (kcal),Protein (g),Carbohydrate (g),"Fiber, total dietary (g)",Total Fat (g),"Fatty acids, total saturated (g)",...,Vitamin K (phylloquinone) (mcg),Calcium (mg),Phosphorus (mg),Magnesium (mg),Iron\n(mg),Zinc\n(mg),Copper (mg),Selenium (mcg),Potassium (mg),Sodium (mg)
0,"Milk, whole",Dairy,0.11,2.4,61,3.27,4.63,0.0,3.20,1.860,...,0.3,123,101,12,0.00,0.42,0.001,1.9,150,38
1,"Milk, reduced fat (2%)",Dairy,0.11,2.4,50,3.36,4.90,0.0,1.90,1.110,...,0.2,126,103,12,0.00,0.43,0.001,1.8,159,39
2,"Milk, low fat (1%)",Dairy,0.11,2.4,43,3.38,5.18,0.0,0.95,0.568,...,0.1,126,103,12,0.00,0.43,0.001,2.1,159,39
3,"Milk, fat free (skim)",Dairy,0.11,2.4,34,3.43,4.92,0.0,0.08,0.049,...,0.0,132,107,12,0.00,0.45,0.002,2.0,167,41
4,"Milk, lactose free, low fat (1%)",Dairy,0.20,2.4,43,3.38,5.18,0.0,0.95,0.568,...,0.1,126,103,12,0.00,0.43,0.001,2.1,159,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,Strawberry drink syrup,Others,1.00,0.5,261,0.00,65.10,0.0,0.02,0.000,...,0.0,0,0,1,0.03,0.06,0.007,0.4,1,0
865,"Pancake syrup, light",Others,3.00,0.5,165,0.00,44.55,0.0,0.00,0.000,...,0.0,10,43,2,0.03,0.20,0.018,0.5,3,178
866,Honey,Others,1.50,0.5,304,0.30,82.40,0.2,0.00,0.000,...,0.0,6,4,2,0.42,0.22,0.036,0.8,52,4
867,Agave liquid sweetener,Others,5.00,0.5,310,0.09,76.37,0.2,0.45,0.000,...,22.5,1,1,1,0.09,0.01,0.009,1.7,4,4


In [322]:
num_vars = len(df.index)

In [323]:
rng = np.random.default_rng(2020)
df['satisfaction'] =  rng.integers(low=-10, high=11, size=num_vars)
df['satisfaction']

0     -10
1      -1
2       5
3       0
4       4
       ..
864     3
865     6
866     3
867     0
868    -6
Name: satisfaction, Length: 869, dtype: int64

In [324]:
requirement = pd.read_csv("requirement.csv")
requirement

,Calories (kcal),Protein (g),Carbohydrate (g),"Fiber, total dietary (g)",Total Fat (g),"Fatty acids, total saturated (g)","Fatty acids, total monounsaturated (g)","Fatty acids, total polyunsaturated (g)",Cholesterol (mg),"Vitamin A, RAE (mcg_RAE)",...,Vitamin K (phylloquinone) (mcg),Calcium (mg),Phosphorus (mg),Magnesium (mg),Iron\n(mg),Zinc\n(mg),Copper (mg),Selenium (mcg),Potassium (mg),Sodium (mg)
0,3294,65,371,38,73,0,0,0,0,900,...,120,1000,700,350,8,11,0.9,55,3400,1500


In [325]:
maximum = pd.read_csv("maximum.csv")
maximum = maximum.fillna(GRB.INFINITY)
maximum

,Calories (kcal),Protein (g),Carbohydrate (g),"Fiber, total dietary (g)",Total Fat (g),"Fatty acids, total saturated (g)","Fatty acids, total monounsaturated (g)","Fatty acids, total polyunsaturated (g)",Cholesterol (mg),"Vitamin A, RAE (mcg_RAE)",...,Vitamin K (phylloquinone) (mcg),Calcium (mg),Phosphorus (mg),Magnesium (mg),Iron\n(mg),Zinc\n(mg),Copper (mg),Selenium (mcg),Potassium (mg),Sodium (mg)
0,1.000000e+100,1.000000e+100,500,1.000000e+100,1.000000e+100,13,44,22,300,3000,...,1.000000e+100,2500,4000,1.000000e+100,45,40,10,400,1.000000e+100,1.000000e+100


In [326]:
max_constr_group = {
    "Dairy": 7.2,
    "Meat": 4,
    "Seafood": 0.6,
    "Grains": 4,
    "Others": 2,
    "Fruits": GRB.INFINITY,
    "Vegetables": GRB.INFINITY

}

min_constr_group = {
    "Dairy": 2.4,
    "Meat": 1.1,
    "Seafood": 0,
    "Grains": 1.7,
    "Others": 0,
    "Fruits": 0.8,
    "Vegetables": 0.8
}

In [327]:
model = grb.Model()

In [328]:
x = model.addVars(num_vars, vtype=GRB.CONTINUOUS, lb=0, ub=3, name="amount")
binary = model.addVars(num_vars, vtype=GRB.BINARY, name="binary")
model.update()
vars = pd.concat([pd.DataFrame.from_dict(binary, orient='index'), pd.DataFrame.from_dict(x, orient='index')], axis=1)
vars.columns = ['binary', 'amount']
vars = pd.concat([vars, df], axis=1)
vars

,binary,amount,Main food description,Category,Price,Maximum_amount,Calories (kcal),Protein (g),Carbohydrate (g),"Fiber, total dietary (g)",...,Calcium (mg),Phosphorus (mg),Magnesium (mg),Iron\n(mg),Zinc\n(mg),Copper (mg),Selenium (mcg),Potassium (mg),Sodium (mg),satisfaction
0,<gurobi.Var binary[0]>,<gurobi.Var amount[0]>,"Milk, whole",Dairy,0.11,2.4,61,3.27,4.63,0.0,...,123,101,12,0.00,0.42,0.001,1.9,150,38,-10
1,<gurobi.Var binary[1]>,<gurobi.Var amount[1]>,"Milk, reduced fat (2%)",Dairy,0.11,2.4,50,3.36,4.90,0.0,...,126,103,12,0.00,0.43,0.001,1.8,159,39,-1
2,<gurobi.Var binary[2]>,<gurobi.Var amount[2]>,"Milk, low fat (1%)",Dairy,0.11,2.4,43,3.38,5.18,0.0,...,126,103,12,0.00,0.43,0.001,2.1,159,39,5
3,<gurobi.Var binary[3]>,<gurobi.Var amount[3]>,"Milk, fat free (skim)",Dairy,0.11,2.4,34,3.43,4.92,0.0,...,132,107,12,0.00,0.45,0.002,2.0,167,41,0
4,<gurobi.Var binary[4]>,<gurobi.Var amount[4]>,"Milk, lactose free, low fat (1%)",Dairy,0.20,2.4,43,3.38,5.18,0.0,...,126,103,12,0.00,0.43,0.001,2.1,159,39,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,<gurobi.Var binary[864]>,<gurobi.Var amount[864]>,Strawberry drink syrup,Others,1.00,0.5,261,0.00,65.10,0.0,...,0,0,1,0.03,0.06,0.007,0.4,1,0,3
865,<gurobi.Var binary[865]>,<gurobi.Var amount[865]>,"Pancake syrup, light",Others,3.00,0.5,165,0.00,44.55,0.0,...,10,43,2,0.03,0.20,0.018,0.5,3,178,6
866,<gurobi.Var binary[866]>,<gurobi.Var amount[866]>,Honey,Others,1.50,0.5,304,0.30,82.40,0.2,...,6,4,2,0.42,0.22,0.036,0.8,52,4,3
867,<gurobi.Var binary[867]>,<gurobi.Var amount[867]>,Agave liquid sweetener,Others,5.00,0.5,310,0.09,76.37,0.2,...,1,1,1,0.09,0.01,0.009,1.7,4,4,0


### Model 1: Minimizing Costs (Satisfaction Score is not included)

In [329]:
expr = vars.apply(lambda x: x['amount'] * x['Price'], axis=1).sum()
model.setObjective(sense=grb.GRB.MINIMIZE, expr=expr)

In [330]:
nutritions = maximum.columns
for nutrition in nutritions:
    lhs = sum(vars['amount'].iloc[i] * df[nutrition].values[i] for i in range(num_vars))
    rhs1 = requirement[nutrition][0]
    rhs2 = maximum[nutrition][0]
    model.addConstr(lhs >= rhs1)
    model.addConstr(lhs <= rhs2)

In [331]:
# maximum quantity constraint
for i in range(num_vars):
    model.addConstr(vars['amount'].iloc[i] <= vars['Maximum_amount'].values[i], name='max_amount' )

In [332]:
# #big-M 
# M = 200
# eps = 0.0001
# for i in range(num_vars):
#     model.addConstr(x[i] >= 0.2 + eps - M * (1 - binary[i]))
#     model.addConstr(x[i] <= M * binary[i])
# model.update()


In [333]:
group = vars.groupby('Category')['amount'].sum()

for category in group.index:
    model.addConstr(group[category] >= min_constr_group[category])
    model.addConstr(group[category] <= max_constr_group[category])
model.update()

In [334]:
# group = vars.groupby('Category')['binary'].sum()
# for category in group.index:
#     if category != "Others":
#         model.addConstr(group[category] >= 1)
# model.update()

In [335]:
model.optimize()
model.write("cost_1_obj.lp")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 943 rows, 1738 columns and 48077 nonzeros
Model fingerprint: 0x1b21fc00
Variable types: 869 continuous, 869 integer (869 binary)
Coefficient statistics:
  Matrix range     [1e-03, 6e+03]
  Objective range  [9e-02, 2e+01]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-01, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 906 rows and 903 columns
Presolve time: 0.06s
Presolved: 37 rows, 850 columns, 22710 nonzeros
Variable types: 850 continuous, 0 integer (0 binary)

Root relaxation: objective 4.348153e+00, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0

In [336]:
print("Total cost is %g" % model.ObjVal)

Total cost is 4.34815


In [337]:
res = pd.DataFrame.from_dict(x, orient='index')
res.columns = ['value']
res.value = res.value.apply(lambda x: x.X)
res = pd.concat([df["Main food description"], res], axis=1)
res[res.value>0]

,Main food description,value
2,"Milk, low fat (1%)",2.400000
3,"Milk, fat free (skim)",2.400000
57,"Strawberry milk, fat free",2.400000
257,Chicken drumstick,0.426282
373,"Egg, whole, raw",0.500000
374,"Egg, white only, raw",0.300000
381,Kidney beans,0.500000
382,"Peruvian beans, from dried",0.500000
384,Mung beans,0.195633
385,Baked beans,0.500000


### Model 2: Satisfaction Score is the Objective Function

#### Objective Function 1: Binary * Satisfaction

In [338]:
model2 = grb.Model("satisfaction_1_obj")

In [339]:
x = model2.addVars(num_vars, vtype=GRB.CONTINUOUS, lb=0, ub=3, name="amount")
binary = model2.addVars(num_vars, vtype=GRB.BINARY, name="binary")
model2.update()
vars = pd.concat([pd.DataFrame.from_dict(binary, orient='index'), pd.DataFrame.from_dict(x, orient='index')], axis=1)
vars.columns = ['binary', 'amount']
vars = pd.concat([vars, df], axis=1)
vars.columns

Index(['binary', 'amount', 'Main food description', 'Category', 'Price',
       'Maximum_amount', 'Calories (kcal)', 'Protein (g)', 'Carbohydrate (g)',
       'Fiber, total dietary (g)', 'Total Fat (g)',
       'Fatty acids, total saturated (g)',
       'Fatty acids, total monounsaturated (g)',
       'Fatty acids, total polyunsaturated (g)', 'Cholesterol (mg)',
       'Vitamin A, RAE (mcg_RAE)', 'Thiamin (mg)', 'Riboflavin (mg)',
       'Niacin (mg)', 'Vitamin B-6 (mg)', 'Folate, total (mcg)',
       'Choline, total (mg)', 'Vitamin B-12 (mcg)', 'Vitamin C (mg)',
       'Vitamin D (D2 + D3) (mcg)', 'Vitamin E (alpha-tocopherol) (mg)',
       'Vitamin K (phylloquinone) (mcg)', 'Calcium (mg)', 'Phosphorus (mg)',
       'Magnesium (mg)', 'Iron\n(mg)', 'Zinc\n(mg)', 'Copper (mg)',
       'Selenium (mcg)', 'Potassium (mg)', 'Sodium (mg)', 'satisfaction'],
      dtype='object')

In [340]:
# add objective function
expr = vars.apply(lambda x: x['binary'] * x['satisfaction'], axis=1).sum()
model2.setObjective(sense=grb.GRB.MAXIMIZE, expr=expr)

In [341]:
# add cost constraint
model2.addConstr(vars.apply(lambda x: x['amount'] * x['Price'], axis=1).sum() <= 9)

# add nutrition value constraints
nutritions = maximum.columns
for nutrition in nutritions:
    lhs = sum(vars['amount'].iloc[i] * df[nutrition].values[i] for i in range(num_vars))
    rhs1 = requirement[nutrition][0]
    rhs2 = maximum[nutrition][0]
    model2.addConstr(lhs >= rhs1)
    model2.addConstr(lhs <= rhs2)

# maximum quantity constraint
for i in range(num_vars):
    model2.addConstr(vars['amount'].iloc[i] <= vars['Maximum_amount'].values[i], name='max_amount')

# big-M 
M = 200
for i in range(num_vars):
    model2.addConstr(x[i] >= 0.001 - M * (1 - binary[i]))
    model2.addConstr(x[i] <= M * binary[i])

group = vars.groupby('Category')['amount'].sum()

for category in group.index:
    model2.addConstr(group[category] >= min_constr_group[category])
    model2.addConstr(group[category] <= max_constr_group[category])

# group = vars.groupby('Category')['binary'].sum()
# for category in group.index:
#     if category != "Others":
#         model2.addConstr(group[category] >= 1)
        
model2.update()

In [342]:
model2.optimize()
model2.write("sat_1_obj.lp")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2682 rows, 1738 columns and 52422 nonzeros
Model fingerprint: 0x3f8774d3
Variable types: 869 continuous, 869 integer (869 binary)
Coefficient statistics:
  Matrix range     [1e-03, 6e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-01, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 0 rows and 1 columns
Presolve removed 905 rows and 0 columns
Presolve time: 0.07s
Presolved: 1777 rows, 1753 columns, 27815 nonzeros
Variable types: 884 continuous, 869 integer (869 binary)

Root relaxation: objective 2.377000e+03, 665 iterations, 0.01 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent  

In [343]:
model2.ObjVal

2377.0

In [344]:
res = pd.DataFrame.from_dict(x, orient='index')
res.columns = ['value']
res.value = res.value.apply(lambda x: x.X)
res = pd.concat([df[["Main food description", 'satisfaction']], res], axis=1)
res[res.value>0]

,Main food description,satisfaction,value
2,"Milk, low fat (1%)",5,0.001000
3,"Milk, fat free (skim)",0,2.400000
4,"Milk, lactose free, low fat (1%)",4,0.001000
5,"Milk, lactose free, fat free (skim)",8,2.400000
7,"Milk, lactose free, whole",5,0.001000
...,...,...,...
860,Corn syrup,9,0.001000
861,Blueberry syrup,4,0.001000
864,Strawberry drink syrup,3,0.438743
865,"Pancake syrup, light",6,0.001000


#### Objective function 2: Amount * Satisfaction score (the more you eat the more you satisfied)

In [345]:
model21 = grb.Model("satisfaction_2_obj")

In [346]:
x = model21.addVars(num_vars, vtype=GRB.CONTINUOUS, lb=0, ub=3, name="amount")
binary = model21.addVars(num_vars, vtype=GRB.BINARY, name="binary")
model21.update()
vars = pd.concat([pd.DataFrame.from_dict(binary, orient='index'), pd.DataFrame.from_dict(x, orient='index')], axis=1)
vars.columns = ['binary', 'amount']
vars = pd.concat([vars, df], axis=1)

In [347]:
# add objective function
expr = vars.apply(lambda x: x['amount'] * x['satisfaction'], axis=1).sum()
model21.setObjective(sense=grb.GRB.MAXIMIZE, expr=expr)

In [348]:
# add cost constraint
model21.addConstr(vars.apply(lambda x: x['amount'] * x['Price'], axis=1).sum() <= 9)

# add nutrition value constraints
nutritions = maximum.columns
for nutrition in nutritions:
    lhs = sum(vars['amount'].iloc[i] * df[nutrition].values[i] for i in range(num_vars))
    rhs1 = requirement[nutrition][0]
    rhs2 = maximum[nutrition][0]
    model21.addConstr(lhs >= rhs1)
    model21.addConstr(lhs <= rhs2)

# maximum quantity constraint
for i in range(num_vars):
    model21.addConstr(vars['amount'].iloc[i] <= vars['Maximum_amount'].values[i], name='max_amount')

#big-M 
# M = 200
# eps = 0.0001
# for i in range(num_vars):
#     model21.addConstr(x[i] >= 0.2 + eps - M * (1 - binary[i]))
#     model21.addConstr(x[i] <= M * binary[i])
    
group = vars.groupby('Category')['amount'].sum()
for category in group.index:
    model21.addConstr(group[category] >= min_constr_group[category])
    model21.addConstr(group[category] <= max_constr_group[category])

# group = vars.groupby('Category')['binary'].sum()
# for category in group.index:
#     if category != "Others":
#         model21.addConstr(group[category] >= 1)
        
model21.update()

In [349]:
model21.optimize()
model21.write("sat_2_obj.lp")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 944 rows, 1738 columns and 48946 nonzeros
Model fingerprint: 0x4efc0135
Variable types: 869 continuous, 869 integer (869 binary)
Coefficient statistics:
  Matrix range     [1e-03, 6e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-01, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 905 rows and 872 columns
Presolve time: 0.06s
Presolved: 39 rows, 880 columns, 24251 nonzeros
Variable types: 880 continuous, 0 integer (0 binary)

Root relaxation: objective 2.586997e+02, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0

In [350]:
res = pd.DataFrame.from_dict(x, orient='index')
res.columns = ['value']
res.value = res.value.apply(lambda x: x.X)
res = pd.concat([df[["Main food description", 'satisfaction']], res], axis=1)
res[res.value>0]

,Main food description,satisfaction,value
5,"Milk, lactose free, fat free (skim)",8,2.400000
11,"Soy milk, light",8,1.095820
46,"Chocolate milk, ready to drink, fat free",4,2.400000
57,"Strawberry milk, fat free",10,2.400000
256,Chicken breast,9,1.159312
265,"Chicken, canned, meat only",10,0.919984
341,"Fish, snapper",-9,0.004675
373,"Egg, whole, raw",5,0.289607
381,Kidney beans,2,0.500000
383,Soybeans,1,0.500000


### Model 3: Diversifying food ingredients

In [351]:
model3 = grb.Model("satisfaction_3_obj")

In [352]:
x = model3.addVars(num_vars, vtype=GRB.CONTINUOUS, lb=0, ub=3, name="amount")
binary = model3.addVars(num_vars, vtype=GRB.BINARY, name="binary")
model3.update()
vars = pd.concat([pd.DataFrame.from_dict(binary, orient='index'), pd.DataFrame.from_dict(x, orient='index')], axis=1)
vars.columns = ['binary', 'amount']
vars = pd.concat([vars, df], axis=1)

In [353]:
# add objective function
expr = vars.apply(lambda x: x['amount'] * x['satisfaction'], axis=1).sum()
model3.setObjective(sense=grb.GRB.MAXIMIZE, expr=expr)

In [354]:
# add cost constraint
model3.addConstr(vars.apply(lambda x: x['amount'] * x['Price'], axis=1).sum() <= 9)

# add nutrition value constraints
nutritions = maximum.columns
for nutrition in nutritions:
    lhs = sum(vars['amount'].iloc[i] * df[nutrition].values[i] for i in range(num_vars))
    rhs1 = requirement[nutrition][0]
    rhs2 = maximum[nutrition][0]
    model3.addConstr(lhs >= rhs1)
    model3.addConstr(lhs <= rhs2)

# maximum quantity constraint
for i in range(num_vars):
    model3.addConstr(vars['amount'].iloc[i] <= vars['Maximum_amount'].values[i], name='max_amount')

# big-M 
M = 200
eps = 0.0001
for i in range(num_vars):
    model3.addConstr(x[i] >= 0.2 + eps - M * (1 - binary[i]))
    model3.addConstr(x[i] <= M * binary[i])
    
group = vars.groupby('Category')['amount'].sum()
for category in group.index:
    model3.addConstr(group[category] >= min_constr_group[category])
    model3.addConstr(group[category] <= max_constr_group[category])

group = vars.groupby('Category')['binary'].sum()
for category in group.index:
    if category != "Others":
        if category not in ["Meat", "Seafood"]:
            model3.addConstr(group[category] >= 2)
        else:
            model3.addConstr(group[category] >= 1)
    
        
model3.update()

In [355]:
model3.optimize()
model3.write("sat_3_obj.lp")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2688 rows, 1738 columns and 53120 nonzeros
Model fingerprint: 0xb1c95e9f
Variable types: 869 continuous, 869 integer (869 binary)
Coefficient statistics:
  Matrix range     [1e-03, 6e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e-01, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 928 rows and 23 columns
Presolve time: 0.08s
Presolved: 1760 rows, 1728 columns, 28967 nonzeros
Variable types: 871 continuous, 857 integer (857 binary)

Root relaxation: objective 2.584266e+02, 176 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

    

In [356]:
res = pd.DataFrame.from_dict(x, orient='index')
res.columns = ['value']
res.value = res.value.apply(lambda x: x.X)
res = pd.concat([df[["Main food description", 'satisfaction']], res], axis=1)
res[res.value>0]

,Main food description,satisfaction,value
5,"Milk, lactose free, fat free (skim)",8,2.400000
11,"Soy milk, light",8,1.083405
27,"Yogurt, liquid",9,0.213014
46,"Chocolate milk, ready to drink, fat free",4,2.186986
57,"Strawberry milk, fat free",10,2.400000
256,Chicken breast,9,1.243549
265,"Chicken, canned, meat only",10,0.410055
341,"Fish, snapper",-9,0.204421
373,"Egg, whole, raw",5,0.335143
381,Kidney beans,2,0.500000
